In [1]:
print("Hello")

Hello


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import timm
from torch.utils.data import DataLoader

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for the training and test sets
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Load CIFAR-100 dataset
train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

# Load pretrained ViT model from timm
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=100)
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(10):  # Adjust the number of epochs as needed
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_accuracy = 100. * correct / total
    print(f'Epoch [{epoch+1}/10], Loss: {running_loss/total:.4f}, Training Accuracy: {train_accuracy:.2f}%')

# Evaluation on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_accuracy = 100. * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')


100%|██████████| 169M/169M [00:07<00:00, 21.6MB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Epoch [1/10], Loss: 0.7883, Training Accuracy: 78.28%
Epoch [2/10], Loss: 0.3590, Training Accuracy: 89.12%
Epoch [3/10], Loss: 0.2562, Training Accuracy: 91.93%
Epoch [4/10], Loss: 0.2069, Training Accuracy: 93.43%
Epoch [5/10], Loss: 0.1666, Training Accuracy: 94.77%
Epoch [6/10], Loss: 0.1576, Training Accuracy: 94.97%
Epoch [7/10], Loss: 0.1247, Training Accuracy: 96.08%
Epoch [8/10], Loss: 0.1110, Training Accuracy: 96.52%
Epoch [9/10], Loss: 0.1126, Training Accuracy: 96.42%
Epoch [10/10], Loss: 0.1035, Training Accuracy: 96.74%
Test Accuracy: 82.79%
